# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD3100 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

## Tarea N° 2: Análisis de embalses y precipitaciones con GEE.

## 1. Introducción.

El análisis geoespacial y la percepción remota juegan un rol crítico en la gestión de recursos hídricos, ya que permiten medir y monitorear la evolución espaciotemporal de fuentes naturales de agua, y de condiciones ambientales como precipitaciones y temperaturas. Además, es posible observar y analizar fenómenos como el crecimiento urbano, industrialización y cambios de uso de suelo, lo cuales determinan los patrones de consumo de agua en una determinada región.

En esta Tarea, utilizaremos datos satelitales disponibles en la plataforma Google Engine para evaluar la evolución de ciertas fuentes de agua y la distribución de precipitaciones en la región del Maule de Chile.
    

## 2. Datos.

Para este ejercicio, trabajaremos con el catálogo de datos de Google Earth Engine.




## Problemas.

### 1. Evolución espacio temporal de embalses en la cuenca del río Maule. [12 ptos]

La cuenca del río Maule abarca prácticamente la totalidad de la Región del Maule, tiene como cauces principales a los ríos Maule, Claro y Loncomilla, y cuenta con infraestructura de embalses y canales riego, y de generación hidroeléctrica. Los dos mayores embalses de agua son las represas Colbún y Laguna del Maule.

El objetivo de este ejercicio es cuantificar la evolución espacio temporal de la superficie de agua de los embalses Colbún y Laguna del Maule entre los años 2017 y 2025, utilizando imágenes satelitales de la misión Sentinel 2.

La detección de cuerpos de agua en base a imagénes multiespectrales puede realizarse por ejemplo en base al índice normalizado de diferencia NDWI (normalized difference water index). Como referencia, vea los siguientes artículos:

https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndwi/ <br>



**1.1[4 ptos]** Genere la geomediana de  Sentinel 2 para el mes de febrero de cada año, con el menor porcentaje de cobertura de nubes posible, y que capture los embalses Colbún-Machicura y la laguna del Maule. <br>
**1.2[4 ptos]** Para cada año, genere una nueva capa de NDWI, y una máscada binaria para identificar los pixeles que corresponde a cuerpos de agua. <br>

**1.3[2 ptos]** Calcule la superficie de agua total ($A_{agua}$) de cada embalse para cada año, y genere un gráfico de serie de tiempo (año vs. $A_{agua}$). Comente, ¿cómo han evolucionado las principales reservas de agua de la región del Maule en los últimos años? <br>

**1.4[2 ptos]** Genere un split map con las capas de NDWI para los años de *menor y mayor* superficie de agua total de los embalses.

Tip: para visualizar las capas de NDWI, puede ser conveniente utilizar los siguiente parámetros:

`visParams = {'min': -1,'max': 1,'palette': ['green', 'blue']}`




#### Solucion

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap,ee
import matplotlib.colors as colors
from shapely.geometry import shape, box
from matplotlib import colors
import matplotlib.patches as mpatches
import contextily as ctx

In [9]:
# Primero determinamos la region del Maule en Chile
# Tomar region desde el geopackage del censo
# lo tomo desde este link: https://censo2024.ine.gob.cl/resultados/ filtrando archivos para la region "Maule"

# ademas, usamos los datos de hidrografia


crs = 32719
hidro = gpd.read_file('hidrografiaChile.gpkg')
print(hidro.crs)
hidro

EPSG:32719


,id,nombre,tipo,escala,nivel,sourcethm,length,geometry
0,1392,None,Quebrada,Local,3,Drenes_02.shp,4830.853,"LINESTRING (375320.942 7544160.73, 375340.975 ..."
1,1384,None,Quebrada,Local,3,Drenes_02.shp,2983.443,"LINESTRING (445299.824 7545891.013, 445296.365..."
2,1385,Quebrada Seca,Quebrada,Regional,1,Drenes_02.shp,2432.783,"LINESTRING (395985.711 7545493.609, 395976.264..."
3,1386,None,Costa,Regional,1,Drenes_02.shp,1329.351,"LINESTRING (373687.708 7543210.927, 373666.541..."
4,1387,None,Quebrada,Local,3,Drenes_02.shp,3630.239,"LINESTRING (389168.438 7545936.413, 389137.534..."
...,...,...,...,...,...,...,...,...
128362,1382,None,Quebrada,Local,3,Drenes_02.shp,2971.270,"LINESTRING (396814.208 7546027.294, 396799.526..."
128363,1383,None,Quebrada,Local,3,Drenes_02.shp,5078.404,"LINESTRING (572275.976 7547844.89, 572264.828 ..."
128364,1388,None,Quebrada,Local,3,Drenes_02.shp,3562.885,"LINESTRING (605690.192 7543081.63, 605750.535 ..."
128365,1469,None,Quebrada,Local,3,Drenes_02.shp,2111.651,"LINESTRING (382894.999 7537569.699, 382944.147..."


In [11]:
hidro['nombre'].value_counts().sort_index()

nombre
Arroyo Agua Dulce         1
Arroyo Aguas Calientes    1
Arroyo Arenas Blancas     1
Arroyo Barrancas          2
Arroyo Barros Negros      1
                         ..
Ventisquero Reicher       8
Zanjon de La Aguada       4
Zanjon de La Guananga     1
Zanjon del Coironal       1
Zanjon del Tofo           1
Name: count, Length: 7716, dtype: int64

### 2. Análisis de patrones de precipitaciones. [6 ptos]

La principal fuente de alimentación de las reservas de agua en la cuenca del Maule son las precipitaciones. Éstas pueden ser registradas mediante sensores, pero la densidad espacial de estaciones metereológicas suele ser baja, por lo que se complementa con estimaciones obtenidas a partir de observaciones satelitales en bandas infra-rojas. Por ejemplo, *Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS)* provee un dataset de precipitaciones cuasi-global desde 1981 al presente, disponibles a través de la plataforma Google Earth Engine. Los datos están disponibles a escala diaria (CHIRPS/DAILY), y a escala de 5 días (CHIRP/PENTAD).

El objetivo de este ejercicio, es estimar las precipitaciones totales obtenidas de  datasets grillados como CHIRP.


**Nota:** Como referencia para este ejercicio, le puede ser de utilidad el siguiente ejemplo:
https://spatialthoughts.com/2020/10/28/rainfall-data-gee/


**2.1[3 ptos]**  De la colección [CHIRPS/PENTAD](https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD), obtenga y visualize un raster con el total de precipitaciones mensuales para el mes de julio 2020 en la región del Maule.

* En GEE, Seleccione las imágenes correspondientes a la región del Maule, para el mes de julio 2020 (son 5 o 6 imágenes). Para esto, filtre la colección de imágenes por fecha (`filterDates`) y región (`filterBounds`).
* Combine (sume) todas las imágenes en un único raster, que representará las precipitaciones totales para el mes estudiado (en mm, para cada pixel).
* Recorte (clip) la imagen total por el polígono de la región del Maule.

**2.2[2 ptos]** Genere una visualización del raster de precipationes totales.

**2.3[1 pto]** Calcule la suma de precipitaciones totales para la región del Maule para el mes de julio 2020.